In [1]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/ptt5-base-portuguese-vocab")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
from datasets import load_dataset

dataset = load_dataset("assin2", cache_dir="dataset/assin2")

In [3]:
train_data = dataset['train']['premise']

In [4]:
#tokenizer = tokenizer.train_new_from_iterator(train_data, vocab_size=1000)

In [5]:
#tokenizer.vocab_size

In [6]:
from transformers import GPTJConfig, GPTJForCausalLM
import json

json_file = open('model/config.json')
config = json.load(json_file)
config['vocab_size'] = tokenizer.vocab_size

config = GPTJConfig(**config)
model = GPTJForCausalLM(config)

In [7]:
#number of parameters

model.num_parameters()

8644964

In [8]:
train_data_tokenized = tokenizer.batch_encode_plus(
    train_data,
    max_length=model.config.n_positions,
    truncation=True,
    padding="max_length",
    return_tensors="pt",
)

In [9]:
import torch

# output
# train_data_tokenized['label'] = train_data_tokenized['input_ids'][:, 1:13]
# train_data_tokenized['label'] = torch.cat((train_data_tokenized['label'], torch.ones(train_data_tokenized['label'].shape[0], 1, dtype=torch.int64)*tokenizer.eos_token_id), dim=1)

# input
# train_data_tokenized['input_ids'] = train_data_tokenized['input_ids'][:, 0:12]
# train_data_tokenized['input_ids'] = torch.cat((train_data_tokenized['input_ids'], torch.ones(train_data_tokenized['input_ids'].shape[0], 1, dtype=torch.int64)*tokenizer.eos_token_id), dim=1)
# train_data_tokenized['attention_mask'] = train_data_tokenized['attention_mask'][:, 0:12]
# train_data_tokenized['attention_mask'] = torch.cat((train_data_tokenized['attention_mask'], torch.ones(train_data_tokenized['attention_mask'].shape[0], 1, dtype=torch.int64)), dim=1)

### training loop

In [13]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

class GPT2Dataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = item['input_ids']
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])


train_dataset = GPT2Dataset(train_data_tokenized)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [9]:
for epoch in range(3):
    model.train()
    loop = tqdm(train_loader)
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/3250 [00:00<?, ?it/s]C:\Users\Matheus\AppData\Local\Temp\ipykernel_2940\602704409.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 0:   0%|          | 1/3250 [00:02<1:56:39,  2.15s/it, loss=10.6]C:\Users\Matheus\AppData\Local\Temp\ipykernel_2940\602704409.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 2: 100%|██████████| 3250/3250 [01:48<00:00, 29.98it/s, loss=0.016]  


In [14]:
# train with trainer

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",  # output directory
    num_train_epochs=3,  # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=2,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir="./logs",  # directory for storing logs
    logging_steps=10,
    fp16=True,
)


trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
)

trainer.train()

  0%|          | 0/9750 [00:00<?, ?it/s]

C:\Users\Matheus\AppData\Local\Temp\ipykernel_16652\64132018.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 10.1773, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.0}
{'loss': 10.128, 'learning_rate': 1.8e-06, 'epoch': 0.01}
{'loss': 10.0409, 'learning_rate': 2.8000000000000003e-06, 'epoch': 0.01}
{'loss': 9.832, 'learning_rate': 3.8e-06, 'epoch': 0.01}
{'loss': 9.5641, 'learning_rate': 4.800000000000001e-06, 'epoch': 0.02}
{'loss': 9.2743, 'learning_rate': 5.8e-06, 'epoch': 0.02}
{'loss': 8.8702, 'learning_rate': 6.800000000000001e-06, 'epoch': 0.02}
{'loss': 8.4936, 'learning_rate': 7.8e-06, 'epoch': 0.02}
{'loss': 8.115, 'learning_rate': 8.8e-06, 'epoch': 0.03}
{'loss': 7.8376, 'learning_rate': 9.800000000000001e-06, 'epoch': 0.03}
{'loss': 7.6477, 'learning_rate': 1.08e-05, 'epoch': 0.03}
{'loss': 7.5351, 'learning_rate': 1.18e-05, 'epoch': 0.04}
{'loss': 7.4531, 'learning_rate': 1.2800000000000001e-05, 'epoch': 0.04}
{'loss': 7.3967, 'learning_rate': 1.3800000000000002e-05, 'epoch': 0.04}
{'loss': 7.34, 'learning_rate': 1.48e-05, 'epoch': 0.05}
{'loss': 7.298, 'learning_rat

C:\Users\Matheus\AppData\Local\Temp\ipykernel_16652\64132018.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 3.9315, 'learning_rate': 4.995675675675676e-05, 'epoch': 0.16}
{'loss': 3.7849, 'learning_rate': 4.9902702702702705e-05, 'epoch': 0.16}
{'loss': 3.608, 'learning_rate': 4.984864864864865e-05, 'epoch': 0.16}
{'loss': 3.4616, 'learning_rate': 4.9794594594594594e-05, 'epoch': 0.17}
{'loss': 3.2995, 'learning_rate': 4.9740540540540545e-05, 'epoch': 0.17}
{'loss': 3.1599, 'learning_rate': 4.968648648648649e-05, 'epoch': 0.17}
{'loss': 3.0057, 'learning_rate': 4.9632432432432434e-05, 'epoch': 0.18}
{'loss': 2.8854, 'learning_rate': 4.957837837837838e-05, 'epoch': 0.18}
{'loss': 2.7252, 'learning_rate': 4.952432432432433e-05, 'epoch': 0.18}
{'loss': 2.5753, 'learning_rate': 4.9470270270270274e-05, 'epoch': 0.18}
{'loss': 2.4222, 'learning_rate': 4.941621621621622e-05, 'epoch': 0.19}
{'loss': 2.2963, 'learning_rate': 4.936216216216216e-05, 'epoch': 0.19}
{'loss': 2.1602, 'learning_rate': 4.930810810810811e-05, 'epoch': 0.19}
{'loss': 2.0064, 'learning_rate': 4.925405405405405e-05, 'ep

...

C:\Users\Matheus\AppData\Local\Temp\ipykernel_16652\64132018.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 0.0588, 'learning_rate': 4.010810810810811e-06, 'epoch': 2.77}
{'loss': 0.0697, 'learning_rate': 3.956756756756757e-06, 'epoch': 2.78}
{'loss': 0.0587, 'learning_rate': 3.9027027027027025e-06, 'epoch': 2.78}
{'loss': 0.0654, 'learning_rate': 3.848648648648649e-06, 'epoch': 2.78}
{'loss': 0.0709, 'learning_rate': 3.794594594594595e-06, 'epoch': 2.78}
{'loss': 0.0897, 'learning_rate': 3.740540540540541e-06, 'epoch': 2.79}
{'loss': 0.0686, 'learning_rate': 3.6864864864864867e-06, 'epoch': 2.79}
{'loss': 0.075, 'learning_rate': 3.6324324324324324e-06, 'epoch': 2.79}
{'loss': 0.0669, 'learning_rate': 3.578378378378378e-06, 'epoch': 2.8}
{'loss': 0.0729, 'learning_rate': 3.5243243243243247e-06, 'epoch': 2.8}
{'loss': 0.0714, 'learning_rate': 3.4702702702702704e-06, 'epoch': 2.8}
{'loss': 0.0688, 'learning_rate': 3.416216216216216e-06, 'epoch': 2.81}
{'loss': 0.0831, 'learning_rate': 3.3621621621621623e-06, 'epoch': 2.81}
{'loss': 0.0782, 'learning_rate': 3.308108108108108e-06, 'epoc

C:\Users\Matheus\AppData\Local\Temp\ipykernel_16652\64132018.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 0.0821, 'learning_rate': 1.308108108108108e-06, 'epoch': 2.93}
{'loss': 0.0771, 'learning_rate': 1.254054054054054e-06, 'epoch': 2.93}
{'loss': 0.0809, 'learning_rate': 1.2000000000000002e-06, 'epoch': 2.93}
{'loss': 0.0816, 'learning_rate': 1.145945945945946e-06, 'epoch': 2.94}
{'loss': 0.0863, 'learning_rate': 1.0918918918918918e-06, 'epoch': 2.94}
{'loss': 0.0758, 'learning_rate': 1.0378378378378377e-06, 'epoch': 2.94}
{'loss': 0.0827, 'learning_rate': 9.837837837837839e-07, 'epoch': 2.94}
{'loss': 0.0733, 'learning_rate': 9.297297297297298e-07, 'epoch': 2.95}
{'loss': 0.079, 'learning_rate': 8.756756756756757e-07, 'epoch': 2.95}
{'loss': 0.064, 'learning_rate': 8.216216216216218e-07, 'epoch': 2.95}
{'loss': 0.0807, 'learning_rate': 7.675675675675676e-07, 'epoch': 2.96}
{'loss': 0.0852, 'learning_rate': 7.135135135135135e-07, 'epoch': 2.96}
{'loss': 0.0706, 'learning_rate': 6.594594594594595e-07, 'epoch': 2.96}
{'loss': 0.0844, 'learning_rate': 6.054054054054055e-07, 'epoch

TrainOutput(global_step=9750, training_loss=0.4920255735654097, metrics={'train_runtime': 298.1368, 'train_samples_per_second': 65.406, 'train_steps_per_second': 32.703, 'train_loss': 0.4920255735654097, 'epoch': 3.0})

### LoRA experiment

In [20]:
# from peft import LoftQConfig, LoraConfig, get_peft_model

# lora_config = LoraConfig()
# peft_model = get_peft_model(model, lora_config)

### Playground

In [21]:
dataset['test']['premise']

['O cachorro caramelo está assistindo um cachorro castanho que está nadando em uma lagoa',
 'O cara está fazendo exercícios no chão',
 'Um cachorro grande e um cachorro pequenino estão parados ao lado do balcão da cozinha e estão investigando',
 'Um menino jovem vestindo um traje de banho vermelho está pulando em uma piscina de crianças azul',
 'Um cara velho com uma barba que é cinza está andando de bicicleta',
 'Uma garrafa está sendo lambida pelo gato',
 'As pessoas não estão andando na estrada ao lado de uma bela cachoeira',
 'Um cara em uma bicicleta está andando em uma roda só',
 'Uma mulher está andando a cavalo',
 'Uma pessoa não está descascando uma banana',
 "Uma criança jovem negra está esperando na beira de um corpo d'água perto de alguns baldes",
 'O monte de homens está brincando com lama em um campo de rúgbi',
 'Quatro pessoas estão andando através da neve espessa e o sol está se pondo',
 'Um bebê está animando outro bebê',
 'Uma mulher está descascando uma batata',
 'Um

In [39]:
index_test = 6
n_words = 2

text = ' '.join(dataset['test']['premise'][index_test].split()[:n_words])
print('original sentence = ', dataset['test']['premise'][index_test])

test_sample = tokenizer.encode(
    text,
    # max_length=model.config.n_positions,
    truncation=True,
    # padding="max_length",
    return_tensors="pt",
    add_special_tokens=False
).to(device)

with torch.no_grad():
    print("input = ", text)
    output = model.generate(input_ids=test_sample, max_length=100)
    print(tokenizer.decode(output[0], skip_special_tokens=True))

original sentence =  As pessoas não estão andando na estrada ao lado de uma bela cachoeira
input =  As pessoas
As pessoas estão sentadas em um campo


In [18]:
test_sample

tensor([[   28, 14793,    21]])